In [ ]:
!cd "/content/drive/MyDrive/Face_recognition/"

In [ ]:
pip install mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 32.2 MB/s 


In [ ]:
pip install imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#
# based on: https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi
#

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np

def init_camera():
  """Create objects and functions in HTML/JavaScript to access local web camera"""

  js = Javascript('''

    // global variables to use in both functions
    var div = null;
    var video = null;   // <video> to display stream from local webcam
    var stream = null;  // stream from local webcam
    var canvas = null;  // <canvas> for single frame from <video> and convert frame to JPG
    var img = null;     // <img> to display JPG after processing with `cv2`

    async function initCamera() {
      // place for video (and eventually buttons)
      div = document.createElement('div');
      document.body.appendChild(div);

      // <video> to display video
      video = document.createElement('video');
      video.style.display = 'block';
      div.appendChild(video);

      // get webcam stream and assing to <video>
      stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;

      // start playing stream from webcam in <video>
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // <canvas> for frame from <video>
      canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      //div.appendChild(input_canvas); // there is no need to display to get image (but you can display it for test)

      // <img> for image after processing with `cv2`
      img = document.createElement('img');
      img.width = video.videoWidth;
      img.height = video.videoHeight;
      div.appendChild(img);
    }

    async function takeImage(quality) {
      // draw frame from <video> on <canvas>
      canvas.getContext('2d').drawImage(video, 0, 0);

      // stop webcam stream
      //stream.getVideoTracks()[0].stop();

      // get data from <canvas> as JPG image decoded base64 and with header "data:image/jpg;base64,"
      return canvas.toDataURL('image/jpeg', quality);
      //return canvas.toDataURL('image/png', quality);
    }

    async function showImage(image) {
      // it needs string "data:image/jpg;base64,JPG-DATA-ENCODED-BASE64"
      // it will replace previous image in `<img src="">`
      img.src = image;
      // TODO: create <img> if doesn't exists, 
      // TODO: use `id` to use different `<img>` for different image - like `name` in `cv2.imshow(name, image)`
    }

  ''')

  display(js)
  eval_js('initCamera()')

def take_frame(quality=0.8):
  """Get frame from web camera"""

  data = eval_js('takeImage({})'.format(quality))  # run JavaScript code to get image (JPG as string base64) from <canvas>

  header, data = data.split(',')  # split header ("data:image/jpg;base64,") and base64 data (JPG)
  data = b64decode(data)  # decode base64
  data = np.frombuffer(data, dtype=np.uint8)  # create numpy array with JPG data

  img = cv2.imdecode(data, cv2.IMREAD_UNCHANGED)  # uncompress JPG data to array of pixels

  return img

def show_frame(img, quality=0.8):
  """Put frame as <img src="data:image/jpg;base64,...."> """

  ret, data = cv2.imencode('.jpg', img)  # compress array of pixels to JPG data

  data = b64encode(data)  # encode base64
  data = data.decode()  # convert bytes to string
  data = 'data:image/jpg;base64,' + data  # join header ("data:image/jpg;base64,") and base64 data (JPG)

  eval_js('showImage("{}")'.format(data))  # run JavaScript code to put image (JPG as string base64) in <img>
                                           # argument in `showImage` needs `" "` 

In [ ]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import cv2
import os
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from numpy import load
from numpy import expand_dims
from numpy import asarray
from keras.models import load_model
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer

from datetime import datetime
import os
import csv

def markAttendance(name): 
    header = ['Name', 'Time']
    now = datetime.now()
    datestring = now.strftime('%y-%m-%d')
    csv_filename = datestring+'.csv'
    if not os.path.exists(csv_filename):
        with open(csv_filename, 'w', encoding='UTF8') as f:
            writer = csv.writer(f)
            # write the header
            writer.writerow(header)
            f.close()

    with open(csv_filename,'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')




# init JavaScript code
init_camera()

def detect_and_predict_mask(frame, faceNet, Clf_Model):
	# grab the dimensions of the frame and then construct a blob
	# from it
	image = frame
	image = Image.fromarray(image)
	# pass the blob through the network and obtain the face detections
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []
	face_Embeddings = []
	# loop over the detections
	for result in results:
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = result['confidence']
		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			# # ensure the bounding boxes fall within the dimensions of
			# # the frame

			box = result['box']
			x1, y1, width, height = box
			x1, y1 = abs(x1), abs(y1)
			x2, y2 = x1 + width, y1 + height
			# extract the face
			face = pixels[y1:y2, x1:x2]
			image = Image.fromarray(face)
			image = image.resize((160,160))
			face = asarray(image)
			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((x1, y1, x2, y2))
	# all the faces of the Image
	faces.extend(faces)
	faces = asarray(faces)

	for f in faces: 
		# scale pixel values
		face_pixels = np.float32(f)
		# standardize pixel values across channels (global)
		mean, std = face_pixels.mean(), face_pixels.std()
		face_pixels = (face_pixels - mean) / std
		# transform face into one sample
		samples = expand_dims(face_pixels, axis=0)
		# make prediction to get embedding
		yhat = faceNet.predict(samples)
		embedding = yhat[0]
		face_Embeddings.append(embedding)

	in_encoder = Normalizer(norm='l2')
	if len(face_Embeddings) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		# faces = np.array(faces, dtype="float32")
		# preds = maskNet.predict(faces, batch_size=32)
		face_Embeddings = asarray(face_Embeddings)
		# normalize input vectors
		face_Embeddings = in_encoder.transform(face_Embeddings)
		preds = Clf_Model.predict(face_Embeddings, batch_size=16)
	# only make a predictions if at least one face was detected

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

# load our serialized face detector model from disk
# base = r'D:\DeepLearning\Udacity\Graduation_project\Face-recognition'
FaceNet_path = "/content/drive/MyDrive/Face_recognition/facenet_keras.h5"

faceNet = load_model(FaceNet_path)
print("FaceNet Loaded")
# load the face embedding classification detector model from disk
Clf_Model = load_model("/content/drive/MyDrive/Face_recognition/Classification.hdf5")
print("Classification model loaded")

# getting our labbels
data = load("/content/drive/MyDrive/Face_recognition/Dataset-faces-embeddings.npz", allow_pickle=True)
trainy=  data['arr_1']
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
# initialize the video stream
print("[INFO] starting video stream...")
# vs = VideoStream(src=0).start()

# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
	frame = take_frame()
	frame = imutils.resize(frame, width=600)
	
	# detect faces in the frame
	(locs, preds) = detect_and_predict_mask(frame, faceNet, Clf_Model)
	
	# loop over the detected face locations and their corresponding
	# locations
	for (box, pred) in zip(locs, preds):
		# unpack the bounding box and predictions
		(startX, startY, endX, endY) = box
		# (stuedent index) = np.argmax(pred)
		class_index = np.argmax(pred)
		
		# determine the class label and color we'll use to draw
		# the bounding box and text
		
		class_probability = pred[class_index] * 100
		predict_names = out_encoder.inverse_transform(np.array([class_index]))


		# color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
		color = (0, 255, 0)
		# include the probability in the label
		label = "{}: {:.2f}%".format(predict_names[0], class_probability)
		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		markAttendance(predict_names[0])
	# show the output frame
	show_frame(frame)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

<IPython.core.display.Javascript object>

FaceNet Loaded
Classification model loaded
[INFO] starting video stream...


KeyboardInterrupt: ignored